In [29]:
import pandas as pd
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora, models
from gensim.utils import simple_preprocess
import gensim, spacy
from gensim.models.ldamulticore import LdaMulticore
import re
from gensim.models import Phrases
from gensim.models.phrases import Phraser

In [30]:
from gsdmm import MovieGroupProcess

In [31]:
df = pd.read_csv('token_lemm.csv')
df

,Unnamed: 0,translated,words_processed_noun,text_type,sentiment,tokenised_stopremoved
0,0,soniya mummy hoti toh covid me magic stick ghu...,['death'],subjective,1,['death']
1,1,Vaccinate to protect yourself and your loved o...,[],subjective,1,[]
2,2,is covid ki waja se buht saray logon ne apne p...,"['hair', 'ply', 'vaccination']",subjective,1,"['hair', 'ply']"
3,3,deaths due covid what is vaccination rate what...,"['death', 'couid', 'vaccination', 'rate', 'har...",subjective,2,"['death', 'rate', 'harm', 'harm']"
4,4,vaccination against covid- 19 is necessity tod...,"['necessity', 'today', 'scenario', 'problem']",subjective,2,"['necessity', 'scenario', 'problem']"
...,...,...,...,...,...,...
43458,43458,nature seems have already started working savi...,"['nature', 'generation', 'life', 'strengthen']",subjective,2,"['nature', 'generation', 'life', 'strengthen']"
43459,43459,"dear , we #india Indiahas helped this #world i...","['world', 'kindnessmatter']",subjective,1,"['world', 'kindnessmatter']"
43460,43460,our sincere request state central govt folded_...,"['request', 'state', 'govt', 'share']",subjective,2,"['request', 'state', 'govt', 'share']"
43461,43461,thanks your appreciation valuable feedback abo...,"['thank', 'appreciation', 'feedback']",subjective,1,"['appreciation', 'feedback']"


In [32]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [33]:
tokenised_lemmatized = list(sent_to_words(df['tokenised_stopremoved']))

In [34]:
tokenised_lemmatized

[['death'],
 [],
 ['hair', 'ply'],
 ['death', 'rate', 'harm', 'harm'],
 ['necessity', 'scenario', 'problem'],
 ['population',
  'yrs',
  'offline',
  'class',
  'examination',
  'situation',
  'safety',
  'class',
  'examination'],
 ['way', 'millennium', 'peril', 'job', 'process'],
 ['symptom', 'example', 'number', 'death', 'population', 'life'],
 ['follower', 'pledge', 'service', 'humanity'],
 ['case', 'case', 'testing', 'taboo'],
 ['intellectual',
  'education',
  'leader',
  'health',
  'system',
  'pandemic',
  'nation',
  'service'],
 ['memory', 'post', 'post', 'forgetting', 'memory', 'couple'],
 ['agreement', 'student', 'travel', 'guideline'],
 ['system',
  'conversion',
  'appeasement',
  'health',
  'service',
  'infection',
  'rate',
  'death',
  'rate',
  'rate',
  'welfare',
  'isis'],
 ['personnel', 'unitedstate'],
 ['oppression', 'appointment', 'wife', 'worker', 'present', 'money'],
 ['oppression', 'appointment', 'wife', 'center', 'worker'],
 ['result',
  'effect',
  'exer

In [35]:
dictionary = gensim.corpora.Dictionary(tokenised_lemmatized)
# create BOW dictionary
bow_corpus = [dictionary.doc2bow(doc) for doc in tokenised_lemmatized]

In [36]:
#Load the save classifier 
import pickle
model_saved = open("india_gsdmm.pickle", "rb") #binary read
model = pickle.load(model_saved)
model_saved.close()

In [37]:
import numpy as np

In [38]:
vocab_length = len(dictionary)

In [39]:
# print number of documents per topic
doc_count = np.array(model.cluster_doc_count)
print('Number of documents per topic :', doc_count)

# Topics sorted by the number of document they are allocated to
top_index = doc_count.argsort()[-15:][::-1]
print('Most important clusters (by number of docs inside):', top_index)

# define function to get top words per topic
def top_words(cluster_word_distribution, top_cluster, values):
    for cluster in top_cluster:
        sort_dicts = sorted(cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]
        print("\nCluster %s : %s"%(cluster, sort_dicts))

# get top words in topics
top_words(model.cluster_word_distribution, top_index, 20)

Number of documents per topic : [12221  2242 10889  4571  9751  3789]
Most important clusters (by number of docs inside): [0 2 4 3 5 1]

Cluster 0 : [('virus', 1169), ('case', 693), ('infection', 627), ('booster', 613), ('shot', 580), ('dose', 544), ('death', 536), ('mask', 454), ('risk', 411), ('life', 399), ('way', 398), ('immunity', 362), ('population', 354), ('world', 354), ('number', 316), ('family', 312), ('patient', 309), ('disease', 301), ('datum', 288), ('doctor', 285)]

Cluster 2 : [('government', 759), ('world', 621), ('state', 545), ('population', 446), ('govt', 396), ('hospital', 320), ('money', 310), ('citizen', 309), ('life', 289), ('price', 277), ('nation', 272), ('leader', 269), ('dose', 264), ('health', 257), ('way', 249), ('company', 220), ('supply', 219), ('center', 216), ('case', 213), ('shortage', 212)]

Cluster 4 : [('dose', 1040), ('hospital', 625), ('center', 546), ('family', 470), ('slot', 463), ('certificate', 453), ('staff', 387), ('parent', 365), ('appointm

In [41]:
# import library from gensim  
from gensim.models import CoherenceModel

# define function to get words in topics
def get_topics_lists(model, top_clusters, n_words):
    '''
    Gets lists of words in topics as a list of lists.
    
    model: gsdmm instance
    top_clusters:  numpy array containing indices of top_clusters
    n_words: top n number of words to include
    
    '''
    # create empty list to contain topics
    topics = []
    
    # iterate over top n clusters
    for cluster in top_clusters:
        #create sorted dictionary of word distributions
        sorted_dict = sorted(model.cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:n_words]
         
        #create empty list to contain words
        topic = []
        
        #iterate over top n words in topic
        for k,v in sorted_dict:
            #append words to topic list
            topic.append(k)
            
        #append topics to topics list    
        topics.append(topic)
    
    return topics

# get topics to feed to coherence model
topics = get_topics_lists(model, top_index, 20) 

# evaluate model using Topic Coherence score
cm_gsdmm = CoherenceModel(topics=topics, 
                          dictionary=dictionary, 
                          corpus=bow_corpus, 
                          texts=tokenised_lemmatized, 
                          coherence='c_v')

# get coherence value
coherence_gsdmm = cm_gsdmm.get_coherence()  

print(coherence_gsdmm)

0.23743701083546817


In [25]:
topics

[['death',
  'virus',
  'rate',
  'case',
  'infection',
  'risk',
  'shot',
  'booster',
  'datum',
  'hospitalization',
  'variant',
  'population',
  'study',
  'immunity',
  'flu',
  'fact',
  'protection',
  'dose',
  'country',
  'disease'],
 ['mask',
  'hospital',
  'mandate',
  'health',
  'care',
  'patient',
  'wear',
  'work',
  'virus',
  'life',
  'choice',
  'risk',
  'case',
  'place',
  'science',
  'rate',
  'folk',
  'death',
  'reason',
  'shot'],
 ['mandate',
  'trucker',
  'country',
  'freedom',
  'government',
  'choice',
  'mask',
  'anthrax',
  'job',
  'protest',
  'life',
  'work',
  'canadian',
  'science',
  'health',
  'death',
  'support',
  'right',
  'state',
  'business'],
 ['government',
  'trust',
  'money',
  'company',
  'science',
  'datum',
  'country',
  'world',
  'trump',
  'dose',
  'drug',
  'shot',
  'stock',
  'fact',
  'work',
  'approval',
  'trial',
  'health',
  'scientist',
  'anthrax'],
 ['shot',
  'booster',
  'arm',
  'dose',
  'si